In [3]:
import pandas as pd
import numpy as np

import datetime
import os, sys
import importlib

import utils
importlib.reload(utils)

from utils import plot_series, plot_series_with_names, plot_series_bar
from utils import plot_dataframe
from utils import get_universe_adjusted_series, scale_weights_to_one, scale_to_book_long_short
from utils import generate_portfolio, backtest_portfolio
from utils import match_implementations

import plotly.graph_objects as go

In [4]:
def backtest_portfolio_new(portfolio: pd.DataFrame, returns: pd.DataFrame, universe: pd.DataFrame, plot_: bool, print_: bool):

    """
    Computes performance metrics from a given portfolio DataFrame.

    This function calculates the net Sharpe ratio, along with other portfolio metrics,
    ensuring that certain constraints are met. It performs checks for:
    - Shape alignment of input DataFrames.
    - portfolio weights only in stocks that are part of the universe.
    - Dollar neutrality.
    - Unit capital constraint.
    - Maximum weight constraint.

    Parameters:
    ----------
    portfolio : pd.DataFrame
        DataFrame representing portfolio over time.
    returns : pd.DataFrame
        DataFrame containing stock returns corresponding to the portfolio.
    universe : pd.DataFrame
        Boolean DataFrame indicating whether a stock is part of the investable universe.
    plot_: bool
        Boolean Flag which decides whether to plot the cumulative PnL
    print_: bool
        Boolean Flag which decides whether to print the metrics of backtest

    Raises:
    ------
    ValueError
        If the input DataFrames do not have matching shapes.
        If portfolio contain stocks that are not in the universe.
        If the portfolio violates the dollar neutrality constraint.
        If the unit capital constraint is violated.
        If the maximum weight constraint is exceeded.

    Returns
    -------
    tuple
        - Net Sharpe Ratio (rounded to 3 decimal places).
        - Pandas Series containing gross PnL over time.

    Additional Outputs:
    -------------------
    - Prints the gross and net Sharpe ratios.
    - Prints the turnover percentage.
    - Plots cumulative Gross and Net PnL.

    Notes:
    ------
    - The turnover is calculated as the average traded capital divided by the average book value.
    - The gross and net Sharpe ratios are annualized using a factor of √252.
    - The net PnL accounts for trading costs (assumed to be 0.01% per unit traded).
    """

    universe = universe.astype(bool)
    
    if not (portfolio.shape == returns.shape == universe.shape):
        raise ValueError("Shapes of portfolio, returns and universe are not algined")

    if ((portfolio.replace(0, np.nan))[~universe].notna().sum().sum() != 0):
        raise ValueError("Your portfolio are present for a stock not present in the universe")

    if ((portfolio.sum(1).abs() > 0.01).sum() > 0):
        raise ValueError("Dollar Neutral Constraint is violated")
    
    if (((portfolio.abs().sum(1) - 1) > 0.01).sum() > 0):
        raise ValueError("Unit Capital Constraint is violated")

    if ((portfolio.abs().max(1) > 0.1).sum() > 0):
        raise ValueError("Maximum Weight Constraint is violated")

    portfolio = portfolio.fillna(0)

    rets = returns.fillna(0)

    gross_pnl = (portfolio * rets).sum(axis=1)

    traded = portfolio.diff(1).abs().sum(axis=1).fillna(0)

    net_pnl = gross_pnl - traded * 1e-4

    gross_sharpe_ratio = (gross_pnl.mean() / gross_pnl.std()) * np.sqrt(252)

    net_sharpe_ratio = (net_pnl.mean() / net_pnl.std()) * np.sqrt(252)

    if print_:
        print("Gross Sharpe Ratio: ", round(gross_sharpe_ratio, 3))
        print("Net Sharpe Ratio: ", round(net_sharpe_ratio, 3))

    return round(net_sharpe_ratio, 3), gross_pnl

In [5]:
data_dir = "/kaggle/input/qrt-quant-quest-iit-bombay-2025/"

features = pd.read_parquet(os.path.join(data_dir, "features.parquet"))

universe = pd.read_parquet(os.path.join(data_dir, "universe.parquet"))
 
returns = pd.read_parquet(os.path.join(data_dir, "returns.parquet"))

In [6]:
def normalize_signal(feature, universe_boolean):
    """
    Normalize a feature signal for a given universe.
    Parameters:
        feature (pd.DataFrame): Feature data for all stocks.
        universe_boolean (pd.DataFrame): Boolean DataFrame indicating tradable stocks.
    Returns:
        pd.DataFrame: Normalized signal.
    """
    signal = feature.shift(1)  # Avoid lookahead bias
    signal = signal.where(universe_boolean, np.nan)
    signal = signal.rank(axis=1, method="min", ascending=True)
    signal = signal.sub(signal.mean(axis=1), axis=0)
    signal = signal.div(signal.abs().sum(axis=1), axis=0)
    return signal


In [7]:
def generate_portfolio_vectorized(features, universe, weights):
    """
    Generate portfolio weights using a vectorized approach.
    Parameters:
        features (pd.DataFrame): DataFrame containing all features for stocks.
        universe (pd.DataFrame): Binary DataFrame indicating tradable stocks.
        weights (array-like): Weights for each feature.
    Returns:
        pd.DataFrame: Portfolio weights for each stock on each trading day.
    """
    universe_boolean = universe.astype(bool)
    features =  features.loc[:'2019']
    # Combine signals into a portfolio
    portfolio_weights = pd.DataFrame(0, index=features.index, columns=features.columns.get_level_values(1).unique())
    
    for i, feature_name in enumerate(features.columns.levels[0]):
        feature_data = features[feature_name]
        normalized_signal = normalize_signal(feature_data, universe_boolean)
        portfolio_weights += normalized_signal * weights[i]
    
    # Enforce dollar neutrality and normalize weights to sum to 1 per day
    portfolio_weights = portfolio_weights.sub(portfolio_weights.mean(axis=1), axis=0)
    portfolio_weights = portfolio_weights.div(portfolio_weights.abs().sum(axis=1), axis=0)
    
    return portfolio_weights.fillna(0)


In [17]:
from scipy.optimize import differential_evolution

# Global variable to track iteration count
iteration_count = [0]  # Use a mutable object to track iterations

def optimize_signal_weights(features, universe, returns):
    def sharpe_ratio_with_penalty(weights):
        """
        Calculate the negative Sharpe ratio with an L2 penalty for optimization.
        """
        # Increment iteration count
        iteration_count[0] += 1
        
        print(f"Debug: Iteration {iteration_count[0]}")
        print(f"Debug: Evaluating Sharpe ratio for weights: {weights}")
        
        # Generate portfolio weights
        portfolio_weights = generate_portfolio_vectorized(features, universe, weights)
        
        # Backtest the portfolio to calculate Sharpe ratio
        sr_vectorized, _ = backtest_portfolio_new(
            portfolio_weights.loc[: "2019"],
            returns.loc[: "2019"],
            universe.loc[: "2019"],
            plot_=False,
            print_=False,
        )
        print(f"Debug: Sharpe ratio calculated: {sr_vectorized}")
        
        # Add L2 penalty to the objective function
        penalty = np.sum(np.square(weights))
        print(f"Debug: L2 penalty: {penalty}")
        
        # Return the negative Sharpe ratio with penalty
        result = -sr_vectorized + 0.0001 * penalty
        print(f"Debug: Objective function value (with penalty): {result}")
        print("-" * 50)
        return result

    # Define bounds for weights
    bounds = [(-10, 10) for _ in range(len(features.columns.levels[0]))]
    print(f"Debug: Bounds for weights: {bounds}")
    
    # Start optimization
    print("Debug: Starting optimization using Differential Evolution...")
    result = differential_evolution(
        sharpe_ratio_with_penalty, 
        bounds,
        mutation=(0.0, 0.1),
        recombination=0.5,
        popsize = 20
    )
    
    # Debugging: Print optimization result
    print(f"Debug: Optimization result: {result}")
    
    # Extract optimal weights and maximum Sharpe ratio
    optimal_weights = result.x
    max_sharpe_ratio = -result.fun
    
    return dict(zip(features.columns.levels[0], optimal_weights)), max_sharpe_ratio

# Optimize weights and calculate maximum Sharpe ratio
print("Debug: Starting weight optimization...")
optimal_signal_weights, max_sharpe_ratio_value = optimize_signal_weights(features, universe, returns)

print("Optimal Signal Weights:", optimal_signal_weights)
print("Maximum Sharpe Ratio:", max_sharpe_ratio_value)

Debug: Starting weight optimization...
Debug: Bounds for weights: [(-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10)]
Debug: Starting optimization using Differential Evolution...
Debug: Iteration 1
Debug: Evaluating Sharpe ratio for weights: [ 0.49954061  6.0004757   0.52368286 -4.12166493 -0.29828299  0.48879403
 -6.32942921  2.01827431  6.58550872  0.87994149  8.5959521  -5.00860571
  2.5502065  -8.90841314 -0.6852818  -2.75307237  4.90356436 -2.42576368
  9.07136549 -6.18283994 -7.25104811  9.83675644]
Debug: Sharpe ratio calculated: -0.871
Debug: L2 penalty: 634.7986854689889
Debug: Objective function value (with penalty): 0.9344798685468989
--------------------------------------------------
Debug: Iteration 2
Debug: Evaluating Sharpe ratio for weights: [ 8.55533087  6.10168838 -9.20658428  6.5

KeyboardInterrupt: 

In [19]:
from deap import base, creator, tools, algorithms
import numpy as np

def optimize_signal_weights(features, universe, returns):
    """
    Optimize weights for signals using a genetic algorithm to maximize Sharpe ratio.
    """
    def sharpe_ratio_with_penalty(weights):
        """
        Calculate the negative Sharpe ratio with an L2 penalty for optimization.
        """
        print(f"Debug: Evaluating Sharpe ratio for weights: {weights}")
        
        # Generate portfolio weights
        portfolio_weights = generate_portfolio_vectorized(features, universe, weights)
        
        # Backtest the portfolio to calculate Sharpe ratio
        sr_vectorized, _ = backtest_portfolio_new(
            portfolio_weights.loc[: "2019"],
            returns.loc[: "2019"],
            universe.loc[: "2019"],
            plot_=False,
            print_=False,
        )
        print(f"Debug: Sharpe ratio calculated: {sr_vectorized}")
        
        # Add L2 penalty to the objective function
        penalty = np.sum(np.square(weights))
        print(f"Debug: L2 penalty: {penalty}")
        
        # Return the negative Sharpe ratio with penalty as a tuple
        result = -sr_vectorized + 0.001 * penalty
        print(f"Debug: Objective function value (with penalty): {result}")
        print("-" * 50)
        return (result,)  # Return as a tuple

    # Define bounds for weights
    bounds = [(-1, 1) for _ in range(len(features.columns.levels[0]))]
    num_weights = len(bounds)

    # Create the fitness function (minimization problem)
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # Create the toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_float", np.random.uniform, -2, 2)  # Random weights in range [-2, 2]
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=num_weights)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Register the evaluation function
    toolbox.register("evaluate", sharpe_ratio_with_penalty)

    # Register genetic algorithm operators
    toolbox.register("mate", tools.cxBlend, alpha=0.5)  # Crossover
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.5, indpb=0.2)  # Mutation
    toolbox.register("select", tools.selTournament, tournsize=3)  # Selection

    # Create the initial population
    population = toolbox.population(n=50)  # Population size

    # Run the genetic algorithm
    print("Debug: Starting optimization using Genetic Algorithm...")
    result_population, logbook = algorithms.eaSimple(
        population,
        toolbox,
        cxpb=0.7,  # Crossover probability
        mutpb=0.2,  # Mutation probability
        ngen=100,  # Number of generations
        verbose=True
    )

    # Extract the best individual
    best_individual = tools.selBest(result_population, k=1)[0]
    optimal_weights = best_individual
    max_sharpe_ratio = -sharpe_ratio_with_penalty(optimal_weights)[0]  # Extract the scalar value from the tuple

    return dict(zip(features.columns.levels[0], optimal_weights)), max_sharpe_ratio

# Optimize weights and calculate maximum Sharpe ratio
print("Debug: Starting weight optimization...")
optimal_signal_weights, max_sharpe_ratio_value = optimize_signal_weights(features, universe, returns)

print("Optimal Signal Weights:", optimal_signal_weights)
print("Maximum Sharpe Ratio:", max_sharpe_ratio_value)

Debug: Starting weight optimization...
Debug: Starting optimization using Genetic Algorithm...
Debug: Evaluating Sharpe ratio for weights: [-1.6535574314616688, -1.1163983164199882, 0.6762587067301999, 0.8509307788375291, 0.3437758230914061, 1.2041725196931772, -1.9047165481056938, -0.38732541374026486, 0.6514751535911714, 1.4476826351420944, -1.2277230941694248, -0.35654868873837264, 0.2210850590660023, -0.8924926910643185, 0.029434929019975797, 0.5405129498232064, -1.3477624491827833, 0.04358071388302687, -1.0243040498614144, -0.22993932215893986, -0.22279897750483935, 1.2612541264546522]


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Debug: Sharpe ratio calculated: -1.262
Debug: L2 penalty: 20.362099241850338
Debug: Objective function value (with penalty): 1.2823620992418503
--------------------------------------------------
Debug: Evaluating Sharpe ratio for weights: [1.5545442055834902, -1.7486611684601656, 0.5653368057544026, 1.252643541890206, -1.8811155315417087, -0.377343583106859, -0.5121067593105475, -1.0391527891256342, -0.07584940311798061, -0.05642268905795689, -0.19748170917361696, 1.7309200079064269, -1.46859542484709, -0.8348134361902093, -1.0349613169932637, 1.3275701164340132, -1.5738874859634726, -0.6789174335286181, 0.7904065148338781, -0.13170609556398372, 0.5716041670102188, 1.6783530552759598]
Debug: Sharpe ratio calculated: 0.144
Debug: L2 penalty: 27.841253835061025
Debug: Objective function value (with penalty): -0.11615874616493896
--------------------------------------------------
Debug: Evaluating Sharpe ratio for weights: [1.5579806854803961, -0.4204008037229561, 0.22711633558242017, -0.

KeyboardInterrupt: 

In [21]:
from deap import base, creator, tools, algorithms
import numpy as np

def optimize_signal_weights(features, universe, returns):
    """
    Optimize weights for signals using a genetic algorithm to maximize Sharpe ratio.
    """
    def sharpe_ratio_with_penalty(weights):
        """
        Calculate the negative Sharpe ratio with an L2 penalty for optimization.
        """
        print(f"Debug: Evaluating Sharpe ratio for weights: {weights}")
        
        # Generate portfolio weights
        portfolio_weights = generate_portfolio_vectorized(features, universe, weights)
        
        # Backtest the portfolio to calculate Sharpe ratio
        sr_vectorized, _ = backtest_portfolio_new(
            portfolio_weights.loc[: "2019"],
            returns.loc[: "2019"],
            universe.loc[: "2019"],
            plot_=False,
            print_=False,
        )
        print(f"Debug: Sharpe ratio calculated: {sr_vectorized}")
        
        # Add L2 penalty to the objective function
        penalty = np.sum(np.square(weights))
        print(f"Debug: L2 penalty: {penalty}")
        
        # Return the negative Sharpe ratio with penalty as a tuple
        result = -sr_vectorized*10 + 0.001 * penalty
        print(f"Debug: Objective function value (with penalty): {result}")
        print("-" * 50)
        return (result,)  # Return as a tuple

    # Define bounds for weights
    bounds = [(-10, 10) for _ in range(len(features.columns.levels[0]))]
    num_weights = len(bounds)

    # Best known weights from previous runs
    global_best_weights = np.array([
        -8.9294198, -0.94113192, -1.9610565, 3.19996843, -9.71325399,
        -5.74400678, 2.17335904, -3.63188841, -6.25243884, 2.54026386,
        4.05747554, -8.05590129, -0.6992199, 9.92685963, 5.97773835,
        -8.72343422, 7.58766015, -1.70724094, 0.73376926, -4.38107197,
        8.77171322, 2.67610819
    ])

    # Create the fitness function (minimization problem)
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # Create the toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_float", np.random.uniform, -10, 10)  # Random weights in range [-2, 2]
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=num_weights)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Initialize a population with the best-known weights as the first individual
    population = toolbox.population(n=50)  # Population size
    population[0][:] = global_best_weights  # Replace the first individual with the best weights

    # Register the evaluation function
    toolbox.register("evaluate", sharpe_ratio_with_penalty)

    # Register genetic algorithm operators
    toolbox.register("mate", tools.cxBlend, alpha=0.5)  # Crossover
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.5, indpb=0.2)  # Mutation
    toolbox.register("select", tools.selTournament, tournsize=3)  # Selection

    # Run the genetic algorithm
    print("Debug: Starting optimization using Genetic Algorithm...")
    result_population, logbook = algorithms.eaSimple(
        population,
        toolbox,
        cxpb=0.7,  # Crossover probability
        mutpb=0.2,  # Mutation probability
        ngen=100,  # Number of generations
        verbose=True
    )

    # Extract the best individual
    best_individual = tools.selBest(result_population, k=1)[0]
    optimal_weights = best_individual
    max_sharpe_ratio = -sharpe_ratio_with_penalty(optimal_weights)[0]  # Extract the scalar value from the tuple

    return dict(zip(features.columns.levels[0], optimal_weights)), max_sharpe_ratio

# Optimize weights and calculate maximum Sharpe ratio
print("Debug: Starting weight optimization...")
optimal_signal_weights, max_sharpe_ratio_value = optimize_signal_weights(features, universe, returns)

print("Optimal Signal Weights:", optimal_signal_weights)
print("Maximum Sharpe Ratio:", max_sharpe_ratio_value)

Debug: Starting weight optimization...
Debug: Starting optimization using Genetic Algorithm...
Debug: Evaluating Sharpe ratio for weights: [-8.9294198, -0.94113192, -1.9610565, 3.19996843, -9.71325399, -5.74400678, 2.17335904, -3.63188841, -6.25243884, 2.54026386, 4.05747554, -8.05590129, -0.6992199, 9.92685963, 5.97773835, -8.72343422, 7.58766015, -1.70724094, 0.73376926, -4.38107197, 8.77171322, 2.67610819]


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Debug: Sharpe ratio calculated: 1.242
Debug: L2 penalty: 742.0544989101933
Debug: Objective function value (with penalty): -11.677945501089807
--------------------------------------------------
Debug: Evaluating Sharpe ratio for weights: [8.026696443958546, 3.2199999712394405, -8.947857533344825, -4.408942784797015, -9.82775829616168, 9.677122825764556, -3.750591327278432, -4.428045701886658, 8.615025879947943, 5.279503937124927, -8.455521539068307, -2.286161337230526, -2.1530026578795747, 7.882641950244768, -6.919851146425602, 5.031024029226458, 1.9856608710238906, 3.233483609957016, 9.835403648724533, -6.083523764616885, -6.646150766547365, -4.294168740645041]
Debug: Sharpe ratio calculated: -0.082
Debug: L2 penalty: 927.740364876962
Debug: Objective function value (with penalty): 1.747740364876962
--------------------------------------------------
Debug: Evaluating Sharpe ratio for weights: [5.570679205494272, 1.6415931669265937, 7.802994178923022, -2.9459382641764726, -2.7398378030

KeyboardInterrupt: 

In [27]:
from bayes_opt import BayesianOptimization
import numpy as np

def optimize_signal_weights_bayesian(features, universe, returns):
    """
    Optimize weights for signals using Bayesian Optimization to maximize Sharpe ratio.
    """
    def sharpe_ratio_objective(**weights):
        """
        Objective function: Negative Sharpe ratio (since we maximize Sharpe).
        """
        # Convert weights dictionary to a numpy array
        weight_array = np.array([weights[key] for key in sorted(weights.keys())])
        
        # Generate portfolio weights
        portfolio_weights = generate_portfolio_vectorized(features, universe, weight_array)
        
        # Backtest the portfolio to calculate Sharpe ratio
        sr_vectorized, _ = backtest_portfolio_new(
            portfolio_weights.loc[: "2019"],
            returns.loc[: "2019"],
            universe.loc[: "2019"],
            plot_=False,
            print_=False,
        )
        print(f"Debug: Sharpe ratio calculated: {sr_vectorized}")
        
        # Add L2 penalty to the objective function
        penalty = np.sum(np.square(weight_array))
        print(f"Debug: L2 penalty: {penalty}")
        
        # Return the negative Sharpe ratio with penalty
        result = sr_vectorized - 0.001 * penalty  # Maximizing Sharpe Ratio
        print(f"Debug: Objective function value (with penalty): {result}")
        print("-" * 50)
        
        return result
    
    # Define search space for weights (-2 to 2 for each feature)
    pbounds = {feature: (-2, 2) for feature in features.columns.levels[0]}
    
    # Initialize Bayesian Optimizer
    optimizer = BayesianOptimization(
        f=sharpe_ratio_objective,
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    
    # Run optimization
    print("Debug: Starting Bayesian Optimization...")
    optimizer.maximize(
        init_points=10,  # Number of random initial points
        n_iter=50,       # Number of optimization iterations
        # acq="ei",        # Acquisition function: Expected Improvement
        # xi=0.01          # Exploration-exploitation trade-off
    )
    
    # Extract best weights
    optimal_weights = optimizer.max['params']
    max_sharpe_ratio = optimizer.max['target']
    
    print("Debug: Optimization Complete")
    print(f"Debug: Best Weights: {optimal_weights}")
    print(f"Debug: Maximum Sharpe Ratio: {max_sharpe_ratio}")
    
    return optimal_weights, max_sharpe_ratio

# Run Bayesian Optimization
optimal_weights, max_sharpe_ratio = optimize_signal_weights_bayesian(features, universe, returns)

print("Optimal Signal Weights:", optimal_weights)
print("Maximum Sharpe Ratio:", max_sharpe_ratio)

Debug: Starting Bayesian Optimization...
|   iter    |  target   | accumu... |   aroon   | averag... | chaiki... | chande... | commod... | ease_o... | ichimoku  | know_s... |   macd    | on_bal... | relati... | stocha... | trend_1_3 | trend_... | trend_... |   trix    | ultima... | volati... | volati... |  volume   | willia... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Debug: Sharpe ratio calculated: -0.55
Debug: L2 penalty: 31.632170143969574
Debug: Objective function value (with penalty): -0.5816321701439696
--------------------------------------------------
| 1         | -0.5816   | -0.5018   | 1.803     | 0.928     | 0.3946    | -1.376    | -1.376    | -1.768    | 1.465     | 0.4045    | 0.8323    | -1.918    | 1.88      | 1.33      | -1

In [33]:
from bayes_opt import BayesianOptimization
import numpy as np
from scipy.stats import loguniform

initial_weights = {
    'accumulation_distribution_index': -10.0,
    'aroon': 10.0,
    'average_true_range': -10.0,
    'chaikin_money_flow': -10.0,
    'chande_momentum_oscillator': -10.0,
    'commodity_channel_index': 10.0,
    'ease_of_movement': 10.0,
    'ichimoku': -10.0,
    'know_sure_thing': -3.63296083186762,
    'macd': -10.0,
    'on_balance_volume': 10.0,
    'relative_strength_index': 10.0,
    'stochastic_oscillator': -10.0,
    'trend_1_3': 9.284263369032148,
    'trend_20_60': -10.0,
    'trend_5_20': -10.0,
    'trix': 6.512698119771214,
    'ultimate_oscillator': -10.0,
    'volatility_20': 10.0,
    'volatility_60': -10.0,
    'volume': 10.0,
    'williams_r': 3.075035263490522
}

def optimize_signal_weights_bayesian(features, universe, returns):
    """
    Optimize weights for signals using Bayesian Optimization to maximize Sharpe ratio.
    """
    def sharpe_ratio_objective(**weights):
        """
        Objective function: Negative Sharpe ratio (since we maximize Sharpe).
        """
        # Convert weights dictionary to a numpy array
        weight_array = np.array([weights[key] for key in sorted(weights.keys())])
        
        # Generate portfolio weights
        portfolio_weights = generate_portfolio_vectorized(features, universe, weight_array)
        
        # Backtest the portfolio to calculate Sharpe ratio
        sr_vectorized, _ = backtest_portfolio_new(
            portfolio_weights.loc[: "2019"],
            returns.loc[: "2019"],
            universe.loc[: "2019"],
            plot_=False,
            print_=False,
        )
        print(f"Debug: Sharpe ratio calculated: {sr_vectorized}")
        
        # Add L2 penalty to the objective function
        penalty = np.sum(np.square(weight_array))
        print(f"Debug: L2 penalty: {penalty}")
        
        # Return the negative Sharpe ratio with penalty
        result = sr_vectorized - 0.0001 * penalty  # Maximizing Sharpe Ratio
        print(f"Debug: Objective function value (with penalty): {result}")
        print("-" * 50)
        
        return result
    
    # Define search space for weights (-2 to 2 for each feature)
    pbounds = {feature: (-10, 10) for feature in features.columns.levels[0]}
    
    # Initialize Bayesian Optimizer
    optimizer = BayesianOptimization(
        f=sharpe_ratio_objective,
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    
    initial_value = sharpe_ratio_objective(**initial_weights)
    optimizer.register(params=initial_weights, target=initial_value)

    # Run optimization
    print("Debug: Starting Bayesian Optimization...")
    optimizer.maximize(
        init_points=20,  # Number of random initial points
        n_iter=100,       # Number of optimization iterations
        # acq="ei",        # Acquisition function: Expected Improvement
        # xi=0.05         # Exploration-exploitation trade-off
    )
    
    # Extract best weights
    optimal_weights = optimizer.max['params']
    max_sharpe_ratio = optimizer.max['target']
    
    print("Debug: Optimization Complete")
    print(f"Debug: Best Weights: {optimal_weights}")
    print(f"Debug: Maximum Sharpe Ratio: {max_sharpe_ratio}")
    
    return optimal_weights, max_sharpe_ratio

# Run Bayesian Optimization
optimal_weights, max_sharpe_ratio = optimize_signal_weights_bayesian(features, universe, returns)

print("Optimal Signal Weights:", optimal_weights)
print("Maximum Sharpe Ratio:", max_sharpe_ratio)

Debug: Sharpe ratio calculated: 2.455
Debug: L2 penalty: 1951.267029382418
Debug: Objective function value (with penalty): 2.259873297061758
--------------------------------------------------
Debug: Starting Bayesian Optimization...
|   iter    |  target   | accumu... |   aroon   | averag... | chaiki... | chande... | commod... | ease_o... | ichimoku  | know_s... |   macd    | on_bal... | relati... | stocha... | trend_1_3 | trend_... | trend_... |   trix    | ultima... | volati... | volati... |  volume   | willia... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Debug: Sharpe ratio calculated: -0.55
Debug: L2 penalty: 790.8042535992395
Debug: Objective function value (with penalty): -0.629080425359924
---------------------------------------------

In [34]:
optimal_weights = {
    'accumulation_distribution_index': -10.0,
    'aroon': -0.09222290817164171,
    'average_true_range': -10.0,
    'chaikin_money_flow': -10.0,
    'chande_momentum_oscillator': 3.318011135314672,
    'commodity_channel_index': 10.0,
    'ease_of_movement': 5.821824288780287,
    'ichimoku': -10.0,
    'know_sure_thing': 10.0,
    'macd': -3.867991947270643,
    'on_balance_volume': 1.8025161396173899,
    'relative_strength_index': 0.6147723614862542,
    'stochastic_oscillator': -10.0,
    'trend_1_3': 10.0,
    'trend_20_60': -10.0,
    'trend_5_20': -10.0,
    'trix': -0.8681094745966318,
    'ultimate_oscillator': 1.3625158710488225,
    'volatility_20': 10.0,
    'volatility_60': -10.0,
    'volume': 10.0,
    'williams_r': -6.987324378181491
}

In [ ]:
from bayes_opt import BayesianOptimization
import numpy as np
from scipy.stats import loguniform

initial_weights = {
    'accumulation_distribution_index': -10.0,
    'aroon': -0.09222290817164171,
    'average_true_range': -10.0,
    'chaikin_money_flow': -10.0,
    'chande_momentum_oscillator': 3.318011135314672,
    'commodity_channel_index': 10.0,
    'ease_of_movement': 5.821824288780287,
    'ichimoku': -10.0,
    'know_sure_thing': 10.0,
    'macd': -3.867991947270643,
    'on_balance_volume': 1.8025161396173899,
    'relative_strength_index': 0.6147723614862542,
    'stochastic_oscillator': -10.0,
    'trend_1_3': 10.0,
    'trend_20_60': -10.0,
    'trend_5_20': -10.0,
    'trix': -0.8681094745966318,
    'ultimate_oscillator': 1.3625158710488225,
    'volatility_20': 10.0,
    'volatility_60': -10.0,
    'volume': 10.0,
    'williams_r': -6.987324378181491
}

def optimize_signal_weights_bayesian(features, universe, returns):
    """
    Optimize weights for signals using Bayesian Optimization to maximize Sharpe ratio.
    """
    def sharpe_ratio_objective(**weights):
        """
        Objective function: Negative Sharpe ratio (since we maximize Sharpe).
        """
        # Convert weights dictionary to a numpy array
        weight_array = np.array([weights[key] for key in sorted(weights.keys())])
        
        # Generate portfolio weights
        portfolio_weights = generate_portfolio_vectorized(features, universe, weight_array)
        
        # Backtest the portfolio to calculate Sharpe ratio
        sr_vectorized, _ = backtest_portfolio_new(
            portfolio_weights.loc[: "2019"],
            returns.loc[: "2019"],
            universe.loc[: "2019"],
            plot_=False,
            print_=False,
        )
        print(f"Debug: Sharpe ratio calculated: {sr_vectorized}")
        
        # Add L2 penalty to the objective function
        penalty = np.sum(np.square(weight_array))
        penalty2 = np.sum(np.abs(weight_array))
        print(f"Debug: L2 penalty: {penalty}")
        print(f"Debug: L1 penalty: {penalty2}")
        
        # Return the negative Sharpe ratio with penalty
        result = sr_vectorized*10 - 0.0007 * penalty - 0.002 * penalty2  # Maximizing Sharpe Ratio
        print(f"Debug: Objective function value (with penalty): {result}")
        print("-" * 50)
        
        return result
    
    # Define search space for weights (-2 to 2 for each feature)
    pbounds = {feature: (-10, 10) for feature in features.columns.levels[0]}
    
    # Initialize Bayesian Optimizer
    optimizer = BayesianOptimization(
        f=sharpe_ratio_objective,
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    
    initial_value = sharpe_ratio_objective(**initial_weights)
    optimizer.register(params=initial_weights, target=initial_value)

    # Run optimization
    print("Debug: Starting Bayesian Optimization...")
    optimizer.maximize(
        init_points=20,  # Number of random initial points
        n_iter=250,       # Number of optimization iterations
        # acq="ei",        # Acquisition function: Expected Improvement
        # xi=0.05         # Exploration-exploitation trade-off
    )
    
    # Extract best weights
    optimal_weights = optimizer.max['params']
    max_sharpe_ratio = optimizer.max['target']
    
    print("Debug: Optimization Complete")
    print(f"Debug: Best Weights: {optimal_weights}")
    print(f"Debug: Maximum Sharpe Ratio: {max_sharpe_ratio}")
    
    return optimal_weights, max_sharpe_ratio

# Run Bayesian Optimization
optimal_weights, max_sharpe_ratio = optimize_signal_weights_bayesian(features, universe, returns)

print("Optimal Signal Weights:", optimal_weights)
print("Maximum Sharpe Ratio:", max_sharpe_ratio)

Debug: Sharpe ratio calculated: 2.521
Debug: L2 penalty: 1414.9324777271488
Debug: L1 penalty: 154.73528850446786
Debug: Objective function value (with penalty): 23.910076688582063
--------------------------------------------------
Debug: Starting Bayesian Optimization...
|   iter    |  target   | accumu... |   aroon   | averag... | chaiki... | chande... | commod... | ease_o... | ichimoku  | know_s... |   macd    | on_bal... | relati... | stocha... | trend_1_3 | trend_... | trend_... |   trix    | ultima... | volati... | volati... |  volume   | willia... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Debug: Sharpe ratio calculated: -0.55
Debug: L2 penalty: 790.8042535992395
Debug: L1 penalty: 117.7198410153806
Debug: Objective function value (wi